In [2]:
import os
import re
from pathlib import Path

import geopandas as gpd
import numpy as np
import xarray as xr
import pandas as pd
from pyprojroot import here
from scipy.spatial import cKDTree

project_root = here()

In [3]:
schism_output_dir = Path("/efs/Zhengtao.Cui/schism_forecast_test/schism_forecast_2025051100/outputs/")  # outputs from the SCHISM test case
outputfiles = sorted(
    schism_output_dir.glob('out2d_*.nc'), 
    key=lambda x: float(re.findall(r"out2d_(\d+)\.nc", x.name)[0])
)

In [4]:
ds = xr.open_mfdataset(outputfiles, engine="netcdf4")

In [5]:
ds

<xarray.Dataset> Size: 52GB
Dimensions:                  (time: 46, one: 1, nSCHISM_hgrid_node: 10537609,
                              nSCHISM_hgrid_face: 20967960,
                              nMaxSCHISM_hgrid_face_nodes: 4,
                              nSCHISM_hgrid_edge: 31505568, two: 2)
Coordinates:
  * time                     (time) float64 368B 3.6e+03 7.2e+03 ... 1.656e+05
Dimensions without coordinates: one, nSCHISM_hgrid_node, nSCHISM_hgrid_face,
                                nMaxSCHISM_hgrid_face_nodes,
                                nSCHISM_hgrid_edge, two
Data variables:
    minimum_depth            (time, one) float64 368B dask.array<chunksize=(1, 1), meta=np.ndarray>
    SCHISM_hgrid_node_x      (time, nSCHISM_hgrid_node) float64 4GB dask.array<chunksize=(1, 10537609), meta=np.ndarray>
    SCHISM_hgrid_node_y      (time, nSCHISM_hgrid_node) float64 4GB dask.array<chunksize=(1, 10537609), meta=np.ndarray>
    depth                    (time, nSCHISM_hgrid_node) float32 2GB dask.array<chunksize=(1, 10537609), meta=np.ndarray>
    bottom_index_node        (time, nSCHISM_hgrid_node) int32 2GB dask.array<chunksize=(1, 10537609), meta=np.ndarray>
    SCHISM_hgrid_face_nodes  (time, nSCHISM_hgrid_face, nMaxSCHISM_hgrid_face_nodes) int32 15GB dask.array<chunksize=(1, 20967960, 4), meta=np.ndarray>
    SCHISM_hgrid_edge_nodes  (time, nSCHISM_hgrid_edge, two) int32 12GB dask.array<chunksize=(1, 31505568, 2), meta=np.ndarray>
    dryFlagNode              (time, nSCHISM_hgrid_node) float32 2GB dask.array<chunksize=(1, 3512537), meta=np.ndarray>
    elevation                (time, nSCHISM_hgrid_node) float32 2GB dask.array<chunksize=(1, 3512537), meta=np.ndarray>
    dryFlagElement           (time, nSCHISM_hgrid_face) float32 4GB dask.array<chunksize=(1, 4193592), meta=np.ndarray>
    dryFlagSide              (time, nSCHISM_hgrid_edge) float32 6GB dask.array<chunksize=(1, 3938196), meta=np.ndarray>

In [6]:
schism_nodes = np.column_stack([ds.isel(time=0).SCHISM_hgrid_node_x.values,ds.isel(time=0).SCHISM_hgrid_node_y.values])

In [7]:
tree = cKDTree(schism_nodes)

In [8]:
project_root

PosixPath('/efs/edfs-deliveries/pi_6/subcase_1/ngwpc-cookbooks/cookbooks/coastal')

In [9]:
cw = gpd.read_file(project_root / "extern/datum-sync/AltGulf_crosswalk.gpkg")
fp = gpd.read_file(project_root / "troute_run_dir/domain/lower_colorado.gpkg", layer="flowpaths")

In [10]:
intersection_points = cw[cw["id"].isin(fp["id"])]

In [11]:
coords = intersection_points[["longitude", "latitude"]].values
distances, indices = tree.query(coords)

In [12]:
schism_subset = ds.isel(nSCHISM_hgrid_node=indices)

#### ------------------------------------------------------
#### Initialize the BMI model
#### ------------------------------------------------------

In [13]:
os.chdir(project_root / "extern/datum-sync")

In [14]:
import yaml
from pathlib import Path
import numpy as np
import pandas as pd
import xarray as xr

from datum_sync.bmi.bmi_datum_sync import BmiDatumSync

data = {
    "crs_input": 4979,  #WGS84 + Height
    "crs_output": 5498,  #NAVD83 + NAVD88
    "z_warn": True,
}
config_file = project_root / "source_sync.yaml"
with open(config_file, 'w') as file:
    yaml.dump(data, file)

model = BmiDatumSync()
model.initialize(config_file)

#### ------------------------------------------------------
#### Process SCHISM dataset time series
#### ------------------------------------------------------

In [15]:
# Extract coordinates (constant across time)
lon_values = schism_subset.SCHISM_hgrid_node_x.isel(time=0).values
lat_values = schism_subset.SCHISM_hgrid_node_y.isel(time=0).values

n_times = schism_subset.time.shape[0]
n_points = len(lon_values)

# Initialize output arrays
converted_coords = np.zeros((n_times, n_points, 3))  # [time, points, [lon,lat,elev]]

In [16]:
print(f"Converting {n_points} nodes across {n_times} time steps...")

# Process each time step
for t in range(n_times):
    # Get elevation values for this time step
    elev_values = schism_subset.elevation.isel(time=t).values
    
    # Set BMI input values
    model.set_value("longitude", lon_values.tolist())
    model.set_value("latitude", lat_values.tolist()) 
    model.set_value("elevation", elev_values.tolist())
    
    # Run conversion
    model.update()
    
    # Get converted coordinates
    dest_array = np.zeros(n_points * 3)
    model.get_value("coordinates__output", dest_array)
    converted_coords[t, :, :] = dest_array.reshape((n_points, 3), order="F")

Converting 7 nodes across 46 time steps...


In [17]:
# ------------------------------------------------------
# Update SCHISM dataset with converted values
# ------------------------------------------------------

schism_converted = schism_subset.copy(deep=True)

# Update coordinates
schism_converted['SCHISM_hgrid_node_x'] = (
    ('time', 'nSCHISM_hgrid_node'), 
    converted_coords[:, :, 0]
)
schism_converted['SCHISM_hgrid_node_y'] = (
    ('time', 'nSCHISM_hgrid_node'), 
    converted_coords[:, :, 1]
)
schism_converted['elevation'] = (
    ('time', 'nSCHISM_hgrid_node'),
    converted_coords[:, :, 2]
)

In [18]:
# Convert Depth
schism_converted['depth'] = np.abs(schism_subset.depth)

In [19]:
# ------------------------------------------------------
# Save results
# ------------------------------------------------------
keep_vars = ['elevation', 'SCHISM_hgrid_node_x', 'SCHISM_hgrid_node_y', 'depth']
schism_minimal = schism_converted[keep_vars]

output_path = project_root / "datum_sync_schism_output.nc"

In [20]:
schism_minimal.to_netcdf(output_path)

print("Finalize")
model.finalize()

Finalize


In [21]:
lon, lat = schism_minimal.SCHISM_hgrid_node_x.isel(time=0).values, schism_minimal.SCHISM_hgrid_node_y.isel(time=0).values

In [22]:
from shapely.geometry import Point

geometry = [Point(x, y) for x, y in zip(lon, lat)]
gdf = gpd.GeoDataFrame(geometry=geometry, crs='EPSG:4326')
gdf.explore()